## DSGE w. Feedback - Numerical Simulations

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib notebook

Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import minimize

In [4]:
import model

## Simulation mechanics

In [39]:
def bound(z: float, gti: float, k: float, p:dict):
    """ Check whether we need to respect the bound on labour"""
    mu = abs(p['rho'])
    x = (z / gti) **  mu - (p['alpha'] / (k ** mu))
    if x < 0:
        return np.infty
    else:
        return ((1 - p['alpha']) / x) ** (1 / mu)


def bisection_CES(z:float, gti:float, k:float, ft_:float, p: dict, init:float=1e3):
    
    print(z, gti, k)
    assert gti > 0, "Negative income"
    
    mu = abs(p['rho'])
    gr = p['gamma'] * (1 + p['interest'])
    kmu = k ** mu
    
    pt1 = -1 * (1 - p['alpha']) * kmu 
    pt2 = 2 * ft_ * z + z
    pt3 = 4 * gti * gr
    pt4 = 2 * p['alpha'] * p['gamma']
    
    def ces(n: float):
        temp = p['alpha'] / kmu + (1 - p['alpha']) / (n ** mu)
        return z * temp ** (1 / mu)
    
    def val(n:float):
        n2 = n **2
        y = ces(n)
        x = pt1 * (pt2 - 2 * p['gamma'] * n2 + pt3 * n2 * y - 2 * gti * y)
        return x + pt4 * n2 * (n ** mu) * (z - 2 * gti * y)
    
    n0 = 1e-3
    b = bound(z, gti, k, p)
    if b == np.infty:
        res = minimize(val, n0, tol=1e-6)
    else:
        bnds = (1e-4, min([init, bound(z, gti, k, p)]))
        res = minimize(val, n0, bounds=[bnds], tol=1e-6)
    
    return res.x[0], ces(res.x[0])

In [40]:
def test_bound():
    p = dict(rho=-5.5, alpha=1/3)
    print("Bound (inf) k=10, gti=1 = {}".format(bound(1, 10, 1, p)))
    print("Bound (0.9289) k=1, gti=10 = {}".format(bound(1, 1, 10, p)))

test_bound()

Bound (inf) k=10, gti=1 = inf
Bound (0.9289) k=1, gti=10 = 0.9289310736134464


In [44]:
def step(t: float, x: np.ndarray, p: dict):
    # Starting variables
    z_, c_, n_, b_, w_, k_, q_, gt_, ft_, news_, inc_, xiz_, xin_ = x
        
    # Random technology process
    rand = np.random.normal(0, p['sigmaZ'])
    xiz = p['etaZ'] * xiz_ + np.sqrt(1 - p['etaZ'] ** 2) * rand
    z = p['zbar'] * np.exp(xiz)
        
    # Income and Investment
    print("w,n",w_,n_)
    print("b,q,k",b_,q_,k_)
    income = (w_ * n_ + b_  + q_ * k_) / (1 + p['inflation'])
    
    # Capital Markets
    k = (1 - p['depreciation']) * k_ + income * (1 - gt_)

    # Household decision
    print(n_, c_)
    n, c = bisection_CES(z, gt_*income, k, ft_, p)
    #temp = (c / z) ** p['rho'] - p['alpha'] * (k ** p['rho'])
    #n = (c / (z*k**p['alpha'])) ** (1 / (1-p['alpha']))
    #n = (temp / (1-p['alpha'])) ** (1 / p['rho'])
    b = (gt_ * income - c) * (1 + p['interest'])
    
    # Firm decisions
    temp = (p['alpha'] * k ** p['rho'] + (1 - p['alpha']) * n ** p['rho']) 
    temp = temp ** ((1 / p['rho']) - 1)
    w = (1 - p['alpha']) * z * temp * (n ** (p['rho'] - 1))
    q = p['alpha'] * z * temp * (k ** (p['rho'] - 1))
    
    #w = (1-p['alpha']) * z * (k / n) ** p['alpha']
    #q = p['alpha'] * z * (n / k) ** (1 - p['alpha'])
    
    # News
    xin = np.random.normal(0, p['sigmaN'])
    info = p['n_cons']*(c/c_ - 1)
    n = p['n_persistence'] * news_ + (1 - p['n_persistence']) * info + xin
    news = np.tanh(p['n_theta'] * n)
    
    if t > 300 and t < 400:
        if p['shock'] == -1:
            news = -1
        elif p['shock'] == 1:
            news = 1
    
    # Household modifiers
    gt = 0.5 * (p['g_max'] + p['g_min'] - news * (p['g_max'] - p['g_min']))
    ft = 0.5 * (p['f_max'] + p['f_min'] - news * (p['f_max'] - p['f_min']))
    
    return z, c, n, b, w, k, q, gt, ft, news, income, xiz, xin

In [45]:
def simulate(start: np.ndarray, p: dict, t_end: float = 1e3):
    x = np.empty((int(t_end), len(start)))
    x[0, :] = start
    for t in range(1, int(t_end)):
        x[t, :] = step(t, x[t - 1, :], p)
    cols = ['z', 'c', 'n', 'b', 'w', 'k', 'q', 'gt', 'ft', 'news', 'income', 'xiz', 'xin']
    df = pd.DataFrame(x, columns=cols)
    df.loc[:,'inv'] = 100*(1-df.loc[:,'gt'])
    df.loc[:,'bc'] = df.b / df.c
    return df

----
## Interactive Analysis

In [47]:
params = {'etaZ': 0.2, 'sigmaZ': 0.8, 'zbar': 1.0, 'sigmaN': 0.2, 
          'inflation': 0.01, 'interest': 0.015, 'depreciation': 0.0, 
          'g_min': 0.5, 'g_max': 0.99, 'f_min': 1.0, 'f_max': 100, 
          'n_cons': 0.2, 'n_theta': 1, 'n_persistence': 0.8, 
          'gamma': 1.0, 'alpha': 0.33, 'rho': -5, 'shock': 1.0}

start = dict(z=1, c=1, n=1, b=1, w=1, k=1, q=0.1, 
             gt=0.8, ft=.5, income=100, news=1, xiz=0, xin=0)
start = np.array([v for _,v in start.items()])

T = 1e2
np.random.seed(40)
view = (1, int(T))


df = simulate(start, params, T)
df = df.iloc[view[0]:view[1],:]

graphs = {
        'ln Consumption & Bonds' : [df.loc[:,['c','b']], True],
        'Utility of Bonds' : [df.ft, False],
        'ln Labour' : [df.n, True],
        'ln Wages' : [df.w, True],
        'ln Income' : [df.income, True],
        'Investment %' : [df.inv, False],
        'ln Capital' : [df.k, True],
        'ln Returns' : [df.q, True],
        'News' : [df.news, False],
        'ln B/C Ratio' : [df.bc, True],
    }


axs = model.simulation_graph(graphs, size = (6, 10))
axs['ln Returns'].axhline(params['interest'], color='red', linewidth=0.5)

w,n 1.0 1.0
b,q,k 1.0 0.1 1.0
1.0 1.0
0.6211279418022426 1.6633663366336635 1.4158415841584158
w,n 0.6053096025567809 79.95781877042664
b,q,k 1.103523227979641 0.03994606416233482 1.4158415841584158
79.95781877042664 0.5761513336980567
0.5316003787446004 24.534314865335745 25.95015644949416
w,n 0.5638380002072693 0.9483010831350751
b,q,k 24.864930086832732 0.005180508640930201 25.95015644949416
0.9483010831350751 0.03684679949068475
0.2076125035264094 14.25714918161454 36.974248142000505
w,n 0.22492500562538856 0.528760929229963
b,q,k 14.404668067108023 2.754866404778508e-08 36.974248142000505
0.528760929229963 0.06535798249333358
4.4040374647170255 9.006269172521442 42.347781584014406
w,n 4.7712920351190995 1.7633179025623988
b,q,k 6.9554992520837775 1.843081182634062e-08 42.347781584014406
1.7633179025623988 2.1535605497788035
2.3472341834535926 7.8212951321682835 49.74312476517945
w,n 2.5429699535129773 0.9384887217308464
b,q,k 7.22362415894808 6.998508554740166e-08 49.7431247651794

<ipython-input-39-d818a419fab8>:33: RuntimeWarning: overflow encountered in multiply
  return x + pt4 * n2 * (n ** mu) * (z - 2 * gti * y)
c:\users\karl\appdata\local\programs\python\python39\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
<ipython-input-39-d818a419fab8>:27: RuntimeWarning: invalid value encountered in power
  return z * temp ** (1 / mu)


AssertionError: Negative income

In [8]:
param_bounds = {
    # Restrict noise params to be constant
    'etaZ': (0.2,0.2), 'sigmaZ': (0.8,0.8), 'sigmaN': (0.2,0.2), 
    # Realistic monthly values - 0% to 5%
    'inflation': (0.0,0.05), 'interest': (0.0,0.05), 'depreciation': (0.0,0.05),
    # Saving and bond parameters
    'g_min': (0.0,0.99), 'g_max': (0.0,0.99), 'f_min': (0.0,1e1), 'f_max': (1e1,1e3),
    # News parameters
    'n_cons': (0.1,10), 'n_theta': (0.1,10), 'n_persistence': (0.0,0.99),
    # Production & Household
    'zbar': (1.0,1.0), 'gamma': (1.0,10.0), 'alpha': (0.1,0.9), 'rho': (-1e4,-1e-1), 
    # No shocks for now
    'shock': (0.0,0.0)}

n = 5
param_steps = {'etaZ': 0, 'sigmaZ': 0, 'sigmaN': 0, 'inflation': 0, 'interest': 0, 'depreciation': 0,
    'g_min': n, 'g_max': n, 'f_min': n, 'f_max': n, 'n_cons': n, 'n_theta': n, 'n_persistence': n,
    'zbar': 0, 'gamma': 0, 'alpha': n, 'rho': n, 'shock': 0}

In [9]:
# Generate the whole set of parameters
params = {'etaZ': 0.2, 'sigmaZ': 0.8, 'zbar': 1.0, 'sigmaN': 0.2, 
          'inflation': 0.01, 'interest': 0.01, 'depreciation': 0.01, 
          'g_min': 0.8, 'g_max': 0.99, 'f_min': 1.0, 'f_max': 100, 
          'n_cons': 1, 'n_theta': 1, 'n_persistence': 0.8, 
          'gamma': 1.0, 'alpha': 0.5, 'rho': -1, 'shock': 0.0}

constants = [k for k,v in param_steps.items() if v==0]
ranges = {k:np.linspace(*v, param_steps[k]) for k,v in param_bounds.items()}
ranges = {k:v for k,v in ranges.items() if v.size!=0}

In [10]:
results = pd.DataFrame(columns = list(ranges.keys()))

In [11]:
from itertools import product

var = list(ranges.keys())

for tup in product(*list(ranges.values())):
    # Generate unique parameter set
    p = dict(params)
    for combo in zip(var, tup):
        p[combo[0]] = combo[1]
    #df = simulate(start)


In [12]:
def dead_economy(df, epsilon):
    return df.c.iloc[-1] < epsilon

In [23]:
# CES
def lhs_CES(cons: float, z: float, k:float, alpha: float, gamma: float, r: float, rho: float) -> float:
        pt1 = 2 * gamma * (1+r) / ((z ** 2) * (1 - alpha) ** (2 / rho))
        pt2 = cons ** 2
        pt3 = (1 - alpha*(z * k / cons) ** rho) ** ((2 - rho) / rho)
        print(pt1, pt2, pt3)
        return pt1 * pt2 * pt3

def rhs_CES(cons: float, gti: float, ft: float) -> float:
    return 1 - (ft * cons) / (gti - cons)
    

def bisection_CES(z:float, income:float, k:float, gt_:float, ft_:float, p:dict, err:float=1e-2)->float:
    assert gt_ * income > 0
    # Define the left and right hand sides of the equations
    diff = lambda c: rhs_CES(c, gt_ * income, ft_) - lhs_CES(c, z, k, p['alpha'], p['gamma'], p['interest'], p['rho'])
    
    # Initial guess at the next options for 
    a = k * z * (p['alpha'] ** (1 / p['rho'])) 
    if a > gt_ * income:
        print("Bad boundaries, setting c=g")
        return gt_ * income
    
    x = [a, 0.5 * (gt_*income + a), gt_*income]
    print(x)
    
    #guess = np.min([gt_*income, k * z * (p['alpha'] ** (1 / p['rho']))])
    #x = [0, guess / 2, guess]
    abs_lst = [abs(diff(i)) for i in x[:2]]
    
    # Apply bi-section method
    while min(abs_lst) >= err:
        test = np.sign([diff(i) for i in x])
        if test[0] == test[1]:
            x = [x[1], (x[1] + x[2]) / 2, x[2]]
        elif test[1] == test[2]:
            x = [x[0], (x[0] + x[1]) / 2, x[1]]

        abs_lst = [abs(diff(i)) for i in x[:2]]

    return x[np.argmin(abs_lst)]